In [1]:
import numpy as np
from scipy.optimize import curve_fit
import scipy.stats
import plotly.graph_objects as pg
import plotly.express as px
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
%load_ext autoreload
%autoreload 2
import dt4dds.analysis as analysis

data = analysis.GroupAnalysis([
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/15c_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/20c_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/25c_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/10c_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/15c_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/20c_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/PCR/25c_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/0a_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/0b_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/2d_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/4d_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/7d_Genscript_GCall")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/0a_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/0b_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/2d_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/4d_Genscript_GCfix")),
    ('Genscript', analysis.ErrorAnalysis("../../data/Aging/7d_Genscript_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/15c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/30c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/45c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/60c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/75c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/90c_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/15c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/30c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/45c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/60c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/75c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/PCR/90c_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/0a_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/0b_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/2d_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/4d_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/7d_Twist_GCall")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/0a_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/0b_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/2d_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/4d_Twist_GCfix")),
    ('Twist', analysis.ErrorAnalysis("../../data/Aging/7d_Twist_GCfix")),
])

# Overall deletion base bias

In [3]:
plot_data = data.data['deletions_by_type'].copy()

def wavg(group):
    d = {}
    d['mean'] = group.ratio.mean()
    d['std'] = group.ratio.std()
    d['count'] = group.ratio.count()
    return pd.Series(d, index=['mean', 'std', 'count'])


df_aggregate = plot_data.groupby(['group', 'type', 'read'], as_index=False).apply(wavg)
df_aggregate

,group,type,read,mean,std,count
0,Genscript,A,1,0.270987,0.031651,17.0
1,Genscript,A,2,0.245503,0.013282,17.0
2,Genscript,C,1,0.252126,0.003236,17.0
3,Genscript,C,2,0.231902,0.015397,17.0
4,Genscript,G,1,0.231739,0.015505,17.0
5,Genscript,G,2,0.251855,0.003458,17.0
6,Genscript,T,1,0.245148,0.013108,17.0
7,Genscript,T,2,0.270740,0.031846,17.0
8,Twist,A,1,0.246854,0.004203,22.0
9,Twist,A,2,0.250685,0.004224,22.0


In [4]:
display(plot_data)

,type,ratio,rate,read,exp,group
0,A,0.303000,0.004017,1,15c_Genscript_GCall,Genscript
1,C,0.250591,0.003322,1,15c_Genscript_GCall,Genscript
2,G,0.215258,0.002854,1,15c_Genscript_GCall,Genscript
3,T,0.231151,0.003065,1,15c_Genscript_GCall,Genscript
4,A,0.231806,0.003067,2,15c_Genscript_GCall,Genscript
...,...,...,...,...,...,...
307,T,0.233631,0.000142,1,7d_Twist_GCfix,Twist
308,A,0.241176,0.000144,2,7d_Twist_GCfix,Twist
309,C,0.268302,0.000160,2,7d_Twist_GCfix,Twist
310,G,0.243588,0.000145,2,7d_Twist_GCfix,Twist


In [5]:
for syn in plot_data.group.unique():
    display(syn)
    window = plot_data.loc[(plot_data.group == syn) & (plot_data.read == "1")]
    model = smf.ols('ratio ~ C(type)', data=window).fit()
    display(model.summary())
    aov = sm.stats.anova_lm(model, typ=2, robust='hc3')
    aov['eta_sq'] = aov[:]['sum_sq']/sum(aov['sum_sq'])
    display(aov)

'Genscript'

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ratio   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     12.76
Date:                Thu, 22 Jun 2023   Prob (F-statistic):           1.23e-06
Time:                        16:28:37   Log-Likelihood:                 175.54
No. Observations:                  68   AIC:                            -343.1
Df Residuals:                      64   BIC:                            -334.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.2710      0.005     59.208      0.000       0.262       0.280
C(type)[T.C]    -0.0189      0.006     -2.914      0.005      -0.032      -0.006
C(type)[T.G]    -0.0392      0.006     -6.064      0.000      -0.052      -0.026
C(type)[T.T]    -0.0258      0.006     -3.992      0.000      -0.039      -0.013
==============================================================================
Omnibus:                        3.792   Durbin-Watson:                   2.072
Prob(Omnibus):                  0.150   Jarque-Bera (JB):                2.008
Skew:                           0.116   Prob(JB):                        0.366
Kurtosis:                       2.191   Cond. No.                         4.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

,sum_sq,df,F,PR(>F),eta_sq
C(type),0.012881,3.0,12.057327,0.000002,0.361099
Residual,0.022791,64.0,NaN,NaN,0.638901


'Twist'

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ratio   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     129.0
Date:                Thu, 22 Jun 2023   Prob (F-statistic):           2.36e-31
Time:                        16:28:38   Log-Likelihood:                 339.18
No. Observations:                  88   AIC:                            -670.4
Df Residuals:                      84   BIC:                            -660.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.2469      0.001    220.653      0.000       0.245       0.249
C(type)[T.C]    -0.0106      0.002     -6.682      0.000      -0.014      -0.007
C(type)[T.G]     0.0201      0.002     12.686      0.000       0.017       0.023
C(type)[T.T]     0.0031      0.002      1.950      0.055   -6.14e-05       0.006
==============================================================================
Omnibus:                        2.358   Durbin-Watson:                   2.544
Prob(Omnibus):                  0.308   Jarque-Bera (JB):                1.693
Skew:                          -0.263   Prob(JB):                        0.429
Kurtosis:                       3.431   Cond. No.                         4.79
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

,sum_sq,df,F,PR(>F),eta_sq
C(type),0.011614,3.0,140.599541,1.208264e-32,0.833926
Residual,0.002313,84.0,NaN,NaN,0.166074


In [6]:
fig = px.bar(
    df_aggregate,
    x="type",
    y="mean",
    error_y="std",
    color="type",
    facet_col="group",
    facet_row="read",
    facet_col_spacing=0.05,
    facet_row_spacing=0.05,
    color_discrete_map={"A": "#3182bd", "C": "#de2d26", "G": "#31a354", "T": "#756bb1"}
)

fig.update_xaxes(
    title='',
    minor_ticks="outside", 
    minor_dtick=0.25,
    title_font_family="Inter",
    title_font_size=28/3, 
    tickfont_size=28/3
)

fig.update_yaxes(
    title='Fraction',
    col=1
)
fig.update_yaxes(
    range=[0, 0.35],
    dtick=0.1,
    minor_ticks="outside", 
    minor_dtick=0.05,
    title_font_family="Inter",
    title_font_size=28/3, 
    tickfont_size=28/3,
)

fig.add_hline(
    y=0.25,
    line_width=2,
    line_dash="dot"
)

fig.update_layout(
    template='simple_white',
    height=330,
    width=330,
    showlegend=False,
    margin=dict(l=0, r=20, t=20, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)

fig.show()
fig.write_image("base_bias.svg")

In [7]:
plot_data = data.data['deletions_by_type'].copy()

def wavg(group):
    d = {}
    d['mean'] = group.ratio.mean()
    d['std'] = group.ratio.std()
    d['count'] = group.ratio.count()
    return pd.Series(d, index=['mean', 'std', 'count'])


df_aggregate = plot_data.groupby(['type', 'read'], as_index=False).apply(wavg)
df_aggregate

,type,read,mean,std,count
0,A,1,0.257373,0.024053,39.0
1,A,2,0.248426,0.009535,39.0
2,C,1,0.243189,0.008704,39.0
3,C,2,0.248922,0.018499,39.0
4,G,1,0.251588,0.020784,39.0
5,G,2,0.243021,0.009224,39.0
6,T,1,0.247850,0.010146,39.0
7,T,2,0.259631,0.023342,39.0


# Deletion base bias by position

In [8]:
plot_data = data.data['deletions_by_refposition_by_type'].copy()

def wavg(group):
    d = {}
    d['mean'] = group.ratio.mean()
    d['std'] = group.ratio.std()
    d['count'] = group.ratio.count()
    return pd.Series(d, index=['mean', 'std', 'count'])


df_aggregate = plot_data.groupby(['group', 'type', 'read', 'position'], as_index=False).apply(wavg)
df_aggregate

,group,type,read,position,mean,std,count
0,Genscript,A,1,0,0.184694,0.033653,17.0
1,Genscript,A,1,1,0.177575,0.005842,17.0
2,Genscript,A,1,2,0.216676,0.005939,17.0
3,Genscript,A,1,3,0.236135,0.009995,17.0
4,Genscript,A,1,4,0.249565,0.017237,17.0
...,...,...,...,...,...,...,...
1795,Twist,T,2,103,0.265583,0.029746,22.0
1796,Twist,T,2,104,0.236886,0.031114,22.0
1797,Twist,T,2,105,0.220025,0.024923,22.0
1798,Twist,T,2,106,0.200266,0.029146,22.0


In [9]:
fig = px.scatter(
    df_aggregate,
    x="position",
    y="mean",
    color="type",
    facet_col="group",
    facet_row="read",
    color_discrete_map={"A": "#3182bd", "C": "#de2d26", "G": "#31a354", "T": "#756bb1"}
)

fig.update_xaxes(
    title='Position',
    row=1,
)
fig.update_xaxes(
    minor_ticks="outside", 
    minor_dtick=25,
    title_font_family="Inter",
    title_font_size=28/3, 
    tickfont_size=28/3
)

fig.update_yaxes(
    title='Fraction',
    col=1
)
fig.update_yaxes(
    dtick=0.25,
    minor_ticks="outside", 
    minor_dtick=0.25/2,
    title_font_family="Inter",
    title_font_size=28/3, 
    tickfont_size=28/3,
)

fig.add_hline(
    y=0.25,
    line_width=2,
    line_dash="dot"
)

fig.update_layout(
    template='simple_white',
    height=330,
    width=330,
    showlegend=False,
    margin=dict(l=0, r=20, t=20, b=0),
    font_family="Inter",
    legend_font_size=28/3,
)

fig.show()
fig.write_image("position_bias.svg")